# Bulk processor and improved FlySight file and jump validation

Uses `./data` as a data lake.

In [ ]:
%%capture
!x=$(pip list | awk '/haversine/') ; [[ -z "$x" ]] && make local

In [ ]:
from pathlib import Path

import warnings

import ipywidgets as widgets
import pandas as pd

In [ ]:
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import getFlySightDataFromCSV
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import processJump
from ssscoring.datatypes import JumpStatus
from ssscoring.constants import FT_IN_M
from ssscoring.flysight import getAllSpeedJumpFilesFrom

---
## Set DZ altitude MSL

Set the value in ft.

In [ ]:
dropZones = {
    'Drop zone': [
        'Bay Area Skydiving',
        'Drop Zone Thailand',
        'Paraclete XP',
        'SkyDance SkyDiving',
        'Skydive Arizona',
        'Skydive Chicago',
        'Thai Sky Adventures',
    ],
    'Alt (ft)': [
        23.0,
        15.0,
        304.0,
        100.0,
        1509.0,
        616.0,
        21.0,
    ],
}

pd.DataFrame(dropZones, columns=[ 'Drop zone', 'Alt (ft)', ])

In [ ]:
dropZoneAltMSL = 15.0
ignoreBaseline = True

In [ ]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

In [ ]:
jumpFiles = getAllSpeedJumpFilesFrom(Path('./data'))

---
## Process and detect non-jump file

The first file in the list for this test is a warm up file.  Detect and ignore it.

In [ ]:
filePath = list(jumpFiles.keys())[0]
rawData, tag = getFlySightDataFromCSV(filePath)
data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
jumpResults = processJump(data)
if jumpResults.status == JumpStatus.WARM_UP_FILE:
    display(widgets.HTML('<h3>%s - warm up file' % filePath.as_posix()))

---
## Process jump file


In [ ]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpFilesList = list(jumpFiles.keys())
if (len(jumpFilesList) > 1):
    filePath = jumpFilesList[1]
    rawData, tag = getFlySightDataFromCSV(filePath)
    data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
    jumpResult = processJump(data)
    if jumpResult.status == JumpStatus.OK:
        display(jumpResult.table)
        display(jumpResult.window)

---
## All files

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue

jumpResults = processAllJumpFiles(jumpFiles=jumpFiles, altitudeDZMeters=dropZoneAltMSLMeters)